In [34]:
using CPUTime
using DelimitedFiles
using Distributed
using Base
using DataStructures
using SharedArrays

In [36]:
numVertex=4
Edges = [(4,1,2), (1,2,1), (2,3,1), (3,4,1), (4,2,3), (1,3,5)]
AdjList = [Vector{}() for _ in 1:numVertex]
for ln in Edges
    push!(AdjList[ln[1]], (ln[2],ln[3]))
    push!(AdjList[ln[2]], (ln[1],ln[3]))
end


In [225]:

function readDataFromFile(filename="in7.txt")
    open(filename, "r") do f 
        n, m = split(readline(f))
        n = parse(Int, n)
        AdjList = [Vector{}() for _ in 1:n]
#         println(n," ",m)
        for ln in eachline(f)
#             a,b,c = split(ln)
            a,b,c = readdlm(IOBuffer(ln), Int)
            if(a==0)
                a=n
            end
            if(b==0)
                b=n
            end
            push!(AdjList[a], (b,c))
            push!(AdjList[b], (a,c))
#             println(a," ",b," ",c)
        end
        return n, AdjList
    end
end
numVertex, AdjList = readDataFromFile();

In [277]:
uf = IntDisjointSets(numVertex+5)


IntDisjointSets([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9996, 9997, 9998, 9999, 10000, 10001, 10002, 10003, 10004, 10005], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 10005)

In [278]:
MAX_INT=2000000000
minCost = SharedArray{Int}(numVertex, init= S -> S[localindices(S)] .= MAX_INT);
minNode = SharedArray{Int}(numVertex, init= S -> S[localindices(S)] .= localindices(S));

In [279]:
mst= Vector{}()
sizehint!(mst, numVertex)


0-element Array{Any,1}

In [280]:
function initMinCostArray()
    for i = 1:numVertex
        minCost[i] = MAX_INT
        minNode[i] = i
    end
end

initMinCostArray (generic function with 1 method)

In [281]:
function findMinCostVertex()
    for i in 1:length(AdjList)
        for j in AdjList[i]
#             println(i," -> ", j[1], "=", j[2], " ",  !in_same_set(uf, i, j[1])  )
            if(minCost[i] > j[2] && !in_same_set(uf, i, j[1]) )
                minCost[i] = j[2]
                minNode[i] = j[1]
            end
        end
    end
end

findMinCostVertex (generic function with 1 method)

In [282]:
res = 0 
mst = Vector{}()

0-element Array{Any,1}

In [283]:

function contractVertex()
    for i in 1:length(AdjList)
        if(!in_same_set(uf, i, minNode[i]))
            union!(uf, i, minNode[i])
            global res+=minCost[i]
            push!(mst, (i, minNode[i], minCost[i]))    
        end
    end
end

contractVertex (generic function with 1 method)

In [284]:
function boruvka_MST()
    i=2
    while(i<numVertex*numVertex && length(mst)< numVertex-1)
        i=i*i
        initMinCostArray()
        findMinCostVertex()
        contractVertex()
        # println(res)
        # println(length(mst))
    end
end

boruvka_MST (generic function with 1 method)

In [285]:
@time @CPUtime boruvka_MST()


elapsed CPU time: 0.516 seconds
  0.515881 seconds (1.97 M allocations: 44.093 MiB)


In [286]:
res

7400292

In [289]:
println(length(mst))

9999
